# 02. StarCraft II Learning Environment (PySC2) - Env, States, Actions (with jupyter notebook)

* 싸이그래머 / DeepStar - PySC2
* 김무성

# 차례
* States (Observations)
* Actions
* How to use like Gym
    - Setting Map
        - Setting Counfigure
        - Create Env
    - Getting state
    - How to act

---------------------------

# States (Observations)

##### 참고자료
* [1] pysc2 environments doc - https://github.com/deepmind/pysc2/blob/master/docs/environment.md


Observations
- https://github.com/deepmind/pysc2/blob/master/docs/environment.md#observation

--------------------------

# Actions

##### 참고자료
* [1] pysc2 environments doc - https://github.com/deepmind/pysc2/blob/master/docs/environment.md
* [2] Train your first Starcraft II learning agent - https://soygema.github.io/starcraftII_machine_learning


Actions - https://github.com/deepmind/pysc2/blob/master/docs/environment.md#actions

<img src="https://soygema.github.io/starcraftII_machine_learning/pysc2_images/pysc2_action.gif" width=600 />

In [ ]:
!python -m pysc2.bin.valid_actions

--------------------------------------

# How to use like Gym

* [3] How to train StarCraft II Bots - https://colab.research.google.com/drive/1AzCKV98UaQQz2aJIeGWlExcxBrpgKsIV
* [4] chagmgang's Reinforcement Learning with StarCraft II - https://drive.google.com/file/d/10641_2vgu937r0FjxxArC1fZTRv5aH1n/view
* [5] Build a Zerg Bot with PySC2 2.0 - https://itnext.io/build-a-zerg-bot-with-pysc2-2-0-295375d2f58e
(github) 
    - How To: PySC2 - https://github.com/skjb/pysc2-tutorial

## Setting Map

### Setting Counfigure

##### 참고자료
* [3] How to train StarCraft II Bots - https://colab.research.google.com/drive/1AzCKV98UaQQz2aJIeGWlExcxBrpgKsIV


In [ ]:
# unfortunately, PySC2 uses Abseil, which treats python code as if its run like an app
# This does not play well with jupyter notebook
# So we will need to monkeypatch sys.argv


import sys
sys.argv = ["python", "--map", "Simple64"]

In [ ]:
from absl import app
from absl import flags

In [ ]:
from pysc2 import maps
from pysc2.env import available_actions_printer
from pysc2.env import run_loop
from pysc2.env import sc2_env
from pysc2.lib import point_flag
from pysc2.lib import stopwatch

In [ ]:
FLAGS = flags.FLAGS

In [ ]:
# because of Abseil's horrible design for running code underneath Colabs
# We have to pull out this ugly hack from the hat
if "flags_defined" not in globals():
    flags.DEFINE_bool("render", True, "Whether to render with pygame.")
    point_flag.DEFINE_point("feature_screen_size", "84",
                            "Resolution for screen feature layers.")
    point_flag.DEFINE_point("feature_minimap_size", "64",
                            "Resolution for minimap feature layers.")
    point_flag.DEFINE_point("rgb_screen_size", None,
                            "Resolution for rendered screen.")
    point_flag.DEFINE_point("rgb_minimap_size", None,
                            "Resolution for rendered minimap.")
    flags.DEFINE_enum("action_space", None, sc2_env.ActionSpace._member_names_,  # pylint: disable=protected-access
                      "Which action space to use. Needed if you take both feature "
                      "and rgb observations.")
    flags.DEFINE_bool("use_feature_units", False,
                      "Whether to include feature units.")
    flags.DEFINE_bool("disable_fog", False, "Whether to disable Fog of War.")

    flags.DEFINE_integer("max_agent_steps", 0, "Total agent steps.")
    flags.DEFINE_integer("game_steps_per_episode", None, "Game steps per episode.")
    flags.DEFINE_integer("max_episodes", 0, "Total episodes.")
    flags.DEFINE_integer("step_mul", 8, "Game steps per agent step.")

    flags.DEFINE_string("agent", "pysc2.agents.random_agent.RandomAgent",
                        "Which agent to run, as a python path to an Agent class.")
    flags.DEFINE_enum("agent_race", "random", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 1's race.")

    flags.DEFINE_string("agent2", "Bot", "Second agent, either Bot or agent class.")
    flags.DEFINE_enum("agent2_race", "random", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 2's race.")
    flags.DEFINE_enum("difficulty", "very_easy", sc2_env.Difficulty._member_names_,  # pylint: disable=protected-access
                      "If agent2 is a built-in Bot, it's strength.")

    flags.DEFINE_bool("profile", False, "Whether to turn on code profiling.")
    flags.DEFINE_bool("trace", False, "Whether to trace the code execution.")
    flags.DEFINE_integer("parallel", 1, "How many instances to run in parallel.")

    flags.DEFINE_bool("save_replay", True, "Whether to save a replay at the end.")

    flags.DEFINE_string("map", None, "Name of a map to use.")
    flags.mark_flag_as_required("map")

In [ ]:
flags_defined = True

In [ ]:
FLAGS(sys.argv)

In [ ]:
players = []
players.append(sc2_env.Agent(sc2_env.Race[FLAGS.agent_race]))
print(players)

In [ ]:
map_name = FLAGS.map
visualize = True
print(map_name)
print(visualize)

### Create Env

In [ ]:
env = sc2_env.SC2Env(
          map_name=map_name,
          players=players,
          agent_interface_format=sc2_env.parse_agent_interface_format(
              feature_screen=FLAGS.feature_screen_size,
              feature_minimap=FLAGS.feature_minimap_size,
              rgb_screen=FLAGS.rgb_screen_size,
              rgb_minimap=FLAGS.rgb_minimap_size,
              action_space=FLAGS.action_space,
              use_feature_units=FLAGS.use_feature_units),
          step_mul=FLAGS.step_mul,
          game_steps_per_episode=FLAGS.game_steps_per_episode,
          disable_fog=FLAGS.disable_fog,
          visualize=visualize)

In [ ]:
env.close()

## Getting state

##### 참고자료
* [4] chagmgang's Reinforcement Learning with StarCraft II - https://drive.google.com/file/d/10641_2vgu937r0FjxxArC1fZTRv5aH1n/view


In [ ]:
env = sc2_env.SC2Env(
          map_name=map_name,
          players=players,
          agent_interface_format=sc2_env.parse_agent_interface_format(
              feature_screen=FLAGS.feature_screen_size,
              feature_minimap=FLAGS.feature_minimap_size,
              rgb_screen=FLAGS.rgb_screen_size,
              rgb_minimap=FLAGS.rgb_minimap_size,
              action_space=FLAGS.action_space,
              use_feature_units=FLAGS.use_feature_units),
          step_mul=FLAGS.step_mul,
          game_steps_per_episode=FLAGS.game_steps_per_episode,
          disable_fog=FLAGS.disable_fog,
          visualize=visualize)

In [ ]:

for episodes in range(10) :
    
    obs = env.reset()
    available_actions = obs[0].observation.available_actions
        
    # -- minimap features    
    minimap_feature = obs[0].observation.feature_minimap
    minimap_height_map = minimap_feature.base[0]
    minimap_visibility_map = minimap_feature.base[1]
    minimap_creep = minimap_feature.base[2]
    minimap_selected = minimap_feature.base[6]
    
    # -- screen features
    screen_feature = obs[0].observation.feature_screen
    screen_height_map = screen_feature.base[0]
    screen_visibility_map = screen_feature.base[1]
    screen_effects = screen_feature.base[16]
        

In [ ]:
env.close()

In [ ]:
def print_val(tag, val) :
    print("========= %s ==========\n" %(tag))
    print(val)

In [ ]:
D_DEBUG = True
if(D_DEBUG) :
    print_val("obs", obs)
    print_val("available_actions", available_actions)
        
    # -- minimap features    
    print_val("minimap_feature", minimap_feature)
    print_val("minimap_height_map", minimap_height_map)
    print_val("minimap_visibility_map", minimap_visibility_map)
    print_val("minimap_creep", minimap_creep)
    print_val("minimap_selected", minimap_selected)
    
    # -- screen features
    print_val("screen_feature", screen_feature)
    print_val("screen_height_map", screen_height_map)
    print_val("screen_visibility_map", screen_visibility_map)
    print_val("screen_effects", screen_effects)

## How to act

##### 참고자료
* [5] Build a Zerg Bot with PySC2 2.0 - https://itnext.io/build-a-zerg-bot-with-pysc2-2-0-295375d2f58e
(github) 
    - How To: PySC2 - https://github.com/skjb/pysc2-tutorial

In [ ]:
from pysc2.agents import random_agent

In [ ]:
import pysc2.agents

In [ ]:
dir(pysc2.agents)

In [ ]:
with sc2_env.SC2Env(
          map_name=map_name,
          players=players,
          agent_interface_format=sc2_env.parse_agent_interface_format(
              feature_screen=FLAGS.feature_screen_size,
              feature_minimap=FLAGS.feature_minimap_size,
              rgb_screen=FLAGS.rgb_screen_size,
              rgb_minimap=FLAGS.rgb_minimap_size,
              action_space=FLAGS.action_space,
              use_feature_units=FLAGS.use_feature_units),
          step_mul=FLAGS.step_mul,
          game_steps_per_episode=FLAGS.game_steps_per_episode,
          disable_fog=FLAGS.disable_fog,
          visualize=visualize) as env :

    agent = random_agent.RandomAgent()
    agent.setup(env.observation_spec()[0], env.action_spec()[0])
    
    done = False
    global_step = 0
    
    obs = env.reset()
    
    while not done : 
        global_step += 1

        #print(obs)
        action = agent.step(obs[0])
        print(action)
        obs = env.step(actions=[action])
       
        if global_step == 10:
            done = True

-----------------------

# 참고자료
* [1] pysc2 environments doc - https://github.com/deepmind/pysc2/blob/master/docs/environment.md
* [2] Train your first Starcraft II learning agent - 
https://soygema.github.io/starcraftII_machine_learning
* [3] How to train StarCraft II Bots - https://colab.research.google.com/drive/1AzCKV98UaQQz2aJIeGWlExcxBrpgKsIV
* [4] chagmgang's Reinforcement Learning with StarCraft II - https://drive.google.com/file/d/10641_2vgu937r0FjxxArC1fZTRv5aH1n/view
* [5] Build a Zerg Bot with PySC2 2.0 - https://itnext.io/build-a-zerg-bot-with-pysc2-2-0-295375d2f58e
     - (github) How To: PySC2 - https://github.com/skjb/pysc2-tutorial